In [4]:
import gdsfactory as gf
import blocks.mylib as mylib
from functools import partial
from gdsfactory.generic_tech import LAYER
from blocks.mylib import waveguide_inv_extrude, adiabatic_resonator, ring_resonator, ruler_set, etch_depth_square, frame
def init_crosssection(core_width, total_width, core_layer='WG', sleeve_layer='DEEP_ETCH'):
    sec1 = gf.Section(
        width=(total_width - core_width) / 2,
        offset=(total_width - core_width) / 4 + core_width / 2,
        layer=sleeve_layer,
    )
    sec2 = gf.Section(
        width=(total_width - core_width) / 2,
        offset=-(total_width - core_width) / 4 - core_width / 2,
        layer=sleeve_layer,
    )
    sec3 = gf.Section(width=core_width, offset=0, layer=core_layer, port_names=["o1", "o2"])
    cs = gf.CrossSection(sections=[sec3,sec1, sec2], radius=10)
    return cs
@gf.cell
def device_array_with_io(device_spec_array, coupler_spec):
    c = gf.Component()
    device_spec_array = list(device_spec_array)
    n = len(device_spec_array)
    for i, devive_spec in enumerate(device_spec_array):
        device_with_io_instance = c << device_with_io(devive_spec, coupler_spec, pitch=500)
        device_with_io_instance.dmovey(i*400-n*200)
    return c

@gf.cell(set_name=False)
def device_with_io(device_spec, coupler_spec, pitch):
    c = gf.Component()
    device = c << device_spec()
    coupler_left = c << coupler_spec()
    coupler_right = c << coupler_spec()
    coupler_left.dmirror_x()
    coupler_left.dmovex(-pitch/2)
    coupler_right.dmovex(pitch/2)
    route_left = gf.routing.route_single(c, coupler_left.ports["o1"], device.ports["o1"],cross_section=cs)
    route_right = gf.routing.route_single(c, coupler_right.ports["o1"], device.ports["o2"],cross_section=cs)
    return c    
    

import numpy as np
from itertools import product
total_width = 5
cs = init_crosssection(0.43, total_width, core_layer='WG', sleeve_layer='DEEP_ETCH')

coupler_spec = partial(gf.components.grating_coupler_elliptical)
my_coupler_spec = partial(mylib.my_coupler,coupler=coupler_spec)

coupler_spec2 = partial(gf.components.grating_coupler_elliptical, wavelength=1.55)
my_coupler_spec2 = partial(mylib.my_coupler,coupler=coupler_spec2)

coupler_spec3 = partial(gf.components.grating_coupler_elliptical, nclad=1)
my_coupler_spec3 = partial(mylib.my_coupler,coupler=coupler_spec3)

coupler_spec4 = partial(gf.components.grating_coupler_elliptical, nclad=1, neff=2.45)
my_coupler_spec4 = partial(mylib.my_coupler,coupler=coupler_spec4)


# left array
instans_adia = [
    partial(waveguide_inv_extrude,
        width=0.43,
        length=50,
        total_width=total_width,
        sleeve_layer="DEEP_ETCH",
        core_layer="WG",
    )
]
instans_adia.extend(
    [ 
        partial(adiabatic_resonator,
            radius=10, angle=10, x_span=50, gap=g, cross_section=cs
        )
        for g in np.linspace(0.15, 0.3, 6)
    ]
)
instans_adia.extend(
    [
        partial(adiabatic_resonator,
            radius=15, angle=10, x_span=50, gap=g, cross_section=cs
        )
        for g in np.linspace(0.15, 0.3, 6)
    ]
)
instans_adia.extend([
    partial(waveguide_inv_extrude,
        width=0.43,
        length=50,
        total_width=total_width,
        sleeve_layer="DEEP_ETCH",
        core_layer="WG",
    )]
)
instans_adia = tuple(instans_adia)

instans_ring = [
        partial(waveguide_inv_extrude,
        width=0.43,
        length=50,
        total_width=total_width,
        sleeve_layer="DEEP_ETCH",
        core_layer="WG",
    )
    ]
instans_ring.extend(
    [
        partial(ring_resonator,cross_section=cs, gap=g, radius=20, length_y=0.1, length_x=3, total_width=total_width,x_span=50)
        for g in np.linspace(0.15, 0.3, 6)
    ]
)
instans_ring.extend(
    [
        partial(ring_resonator,cross_section=cs, gap=g, radius=20, length_y=0.1, length_x=5, total_width=total_width,x_span=50)
        for g in np.linspace(0.15, 0.3, 6)
    ]
)
instans_ring.extend(
    [partial(waveguide_inv_extrude,
        width=0.43,
        length=50,
        total_width=total_width,
        sleeve_layer="DEEP_ETCH",
        core_layer="WG",
    )]
)

instans_ring = tuple(instans_ring)


c = gf.Component()
for i, (_coupler_spec, instans_list) in enumerate(product([my_coupler_spec, my_coupler_spec2, my_coupler_spec3,my_coupler_spec4], [instans_adia, instans_ring])):
    deviceArray = device_array_with_io(instans_list, _coupler_spec)
    (c << deviceArray).dmovex(i*1000-3500)
    
# add row, col labels
n_rows = len(instans_adia)
n_cols = len(list(product([my_coupler_spec, my_coupler_spec2, my_coupler_spec3,my_coupler_spec4], [instans_adia, instans_ring])))
pitch_x = 1000
pitch_y = 400
for i in range(n_rows):
    for j in range(n_cols):
        label = mylib.text_outline(f"({i},{j})",layer=(3,6),with_mask=False,size=50,outline_width=3)
        label = c << label
        label.dmove(((j-n_cols/2)*pitch_x+100, (i-n_rows/2)*pitch_y+50))


ruler_set1 = c << ruler_set()
ruler_set2 = c << ruler_set()
ruler_set3 = c << ruler_set()
ruler_set2.dmovex(2000)
ruler_set3.dmovex(-2000)

depth_square1 = c << etch_depth_square((50,50),(2,6),(3,6))
depth_square2 = c << etch_depth_square((50,50),(2,6),(3,6))
depth_square3 = c << etch_depth_square((50,50),(2,6),(3,6))
depth_square1.dmovey(200)
depth_square2.dmovey(-200)
depth_square3.dmovex(1000)


c << mylib.die_with_alignment_marks(die_size=10000)
c << frame(10000,((2,6),(3,6)))
c.show()
# c.write_gds(r"gds_files\ID55_doubleport.gds",with_metadata=False)

c:\Users\weixi\anaconda3\envs\layout\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 20} ignored for cross_section 'xs_66e883a2'
  x = gf.get_cross_section(cross_section, radius=radius)
